In [96]:
import numpy as np
import pandas as pd
import math

In [98]:
def pairwise_dist(x, y):  # [5 pts]
    """
        Args:
            x: N x D numpy array
            y: M x D numpy array
        Return:
                dist: N x M array, where dist2[i, j] is the euclidean distance between
                x[i, :] and y[j, :]
    """
    #raise NotImplementedError
    return np.linalg.norm(x[:, None, :] - y[None, :, :], axis=-1)

In [114]:
cities = pd.read_csv("data/data_standardized.csv")
cities = cities.rename(columns = {'Unnamed: 0':'city_id'})
cities.head()

,city_id,State Abbreviation,Average Household Income 2020/2021,"Percent Change in Household Income, 1984-2019","Average Yearly Percent Change in Household Income, 1984-2019",MinWage,City,"Percent Change in Housing Prices, 2000-2019","Average Yearly Percent Change in Housing Prices, 2000-2019",Average House Price 2020/2021,...,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,0,AL,-1.244475,0.682348,0.682348,-1.13437,Arab,-1.424617,-1.424617,-0.530148,...,-0.594826,-0.742922,-1.335937,-1.240936,-0.374992,-0.154439,0.004864,0.328125,0.041230,-0.018375
1,1,AL,-1.244475,0.682348,0.682348,-1.13437,Attalla,-0.357814,-0.357814,-0.754485,...,-0.596985,-1.132559,-0.880312,-0.555329,-0.708592,0.379059,-0.150118,-0.545845,0.117471,0.980909
2,2,AL,-1.244475,0.682348,0.682348,-1.13437,Auburn,-0.324323,-0.324323,-0.237935,...,-0.458780,0.425989,-0.465186,-0.974311,-0.977773,-0.483073,1.150718,-1.026759,-0.416213,-0.200487
3,3,AL,-1.244475,0.682348,0.682348,-1.13437,Bay Minette,-0.168164,-0.168164,-0.518289,...,-0.640174,-1.049909,-0.384185,0.619997,0.255398,0.152856,-0.256827,0.184776,-0.797416,-0.256522
4,4,AL,-1.244475,0.682348,0.682348,-1.13437,Chickasaw,-1.612573,-1.612573,-0.756796,...,-0.411272,-0.630754,-0.617061,-0.582536,-0.448614,0.639406,-0.254287,-0.943524,0.269952,1.209716


In [169]:
city_all.columns

Index(['city_id', 'State Abbreviation', 'Average Household Income 2020/2021',
       'Percent Change in Household Income, 1984-2019',
       'Average Yearly Percent Change in Household Income, 1984-2019',
       'MinWage', 'City', 'Percent Change in Housing Prices, 2000-2019',
       'Average Yearly Percent Change in Housing Prices, 2000-2019',
       'Average House Price 2020/2021', 'County Name', 'Violent Crime Total',
       'Property Crime Total', 'Property Crime Per Capita',
       'Violent Crime Per Capita', 'Total Crime Per Capita', 'State', 'County',
       'TotalPop', 'Men', 'Women', 'VotingAgeCitizen', 'Employed', 'Hispanic',
       'White', 'Black', 'Native', 'Asian', 'Pacific', 'Income',
       'IncomePerCap', 'Poverty', 'ChildPoverty', 'Professional', 'Service',
       'Office', 'Construction', 'Production', 'Drive', 'Carpool', 'Transit',
       'Walk', 'OtherTransp', 'WorkAtHome', 'MeanCommute', 'PrivateWork',
       'PublicWork', 'SelfEmployed', 'FamilyWork', 'Unemployme

In [150]:
data_us = pd.read_csv('data/uscities.csv')

data_us = data_us[['city', 'state_id', 'state_name', 'county_name', 'lat', 'lng']]
data_us['city_name_id'] = data_us.apply(lambda x: x['city'] + ", " + x['state_name'] + ", " + x['county_name'] + " County", axis = 1)
data_us['city_state_id'] = data_us.apply(lambda x: x['city'] + ", " + x['state_name'], axis = 1)

data_cities = cities.copy()
data_cities['city_name_id'] = data_cities.apply(lambda x: x['City'] + ", " + x['State'] + ", " + x['County'], axis = 1)
data_cities['city_state_id'] = data_cities.apply(lambda x: x['City'] + ", " + x['State'], axis = 1)

city_all = data_cities.merge(data_us, on = 'city_name_id', how = 'left')
city_all = city_all.rename(columns = {'city_state_id_x':'city_state'})
city_all.dropna(inplace = True)
city_all = city_all.drop(columns = 'city_id')
city_all = city_all.reset_index(drop = True)
city_all = city_all.reset_index()
city_all = city_all.rename(columns = {'index':'city_id'})
city_all.to_csv('data/city_all.csv', index = False)
city_all

,city_id,State Abbreviation,Average Household Income 2020/2021,"Percent Change in Household Income, 1984-2019","Average Yearly Percent Change in Household Income, 1984-2019",MinWage,City,"Percent Change in Housing Prices, 2000-2019","Average Yearly Percent Change in Housing Prices, 2000-2019",Average House Price 2020/2021,...,Unemployment,city_name_id,city_state,city,state_id,state_name,county_name,lat,lng,city_state_id_y
0,0,AL,-1.244475,0.682348,0.682348,-1.134370,Arab,-1.424617,-1.424617,-0.530148,...,-0.018375,"Arab, Alabama, Marshall County","Arab, Alabama",Arab,AL,Alabama,Marshall,34.3309,-86.4991,"Arab, Alabama"
1,1,AL,-1.244475,0.682348,0.682348,-1.134370,Attalla,-0.357814,-0.357814,-0.754485,...,0.980909,"Attalla, Alabama, Etowah County","Attalla, Alabama",Attalla,AL,Alabama,Etowah,34.0050,-86.1039,"Attalla, Alabama"
2,2,AL,-1.244475,0.682348,0.682348,-1.134370,Auburn,-0.324323,-0.324323,-0.237935,...,-0.200487,"Auburn, Alabama, Lee County","Auburn, Alabama",Auburn,AL,Alabama,Lee,32.6087,-85.4903,"Auburn, Alabama"
3,3,AL,-1.244475,0.682348,0.682348,-1.134370,Bay Minette,-0.168164,-0.168164,-0.518289,...,-0.256522,"Bay Minette, Alabama, Baldwin County","Bay Minette, Alabama",Bay Minette,AL,Alabama,Baldwin,30.8930,-87.7912,"Bay Minette, Alabama"
4,4,AL,-1.244475,0.682348,0.682348,-1.134370,Chickasaw,-1.612573,-1.612573,-0.756796,...,1.209716,"Chickasaw, Alabama, Mobile County","Chickasaw, Alabama",Chickasaw,AL,Alabama,Mobile,30.7715,-88.0793,"Chickasaw, Alabama"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3416,3416,WV,-1.767176,0.419941,0.419941,-0.586476,Princeton,-1.049909,-1.049909,-0.695213,...,0.028321,"Princeton, West Virginia, Mercer County","Princeton, West Virginia",Princeton,WV,West Virginia,Mercer,37.3689,-81.0961,"Princeton, West Virginia"
3417,3417,WV,-1.767176,0.419941,0.419941,-0.586476,Shepherdstown,-0.429773,-0.429773,-0.044345,...,0.331841,"Shepherdstown, West Virginia, Jefferson County","Shepherdstown, West Virginia",Shepherdstown,WV,West Virginia,Jefferson,39.4320,-77.8046,"Shepherdstown, West Virginia"
3418,3418,WV,-1.767176,0.419941,0.419941,-0.586476,South Charleston,-1.210938,-1.210938,-0.616320,...,-0.027714,"South Charleston, West Virginia, Kanawha County","South Charleston, West Virginia",South Charleston,WV,West Virginia,Kanawha,38.3426,-81.7155,"South Charleston, West Virginia"
3419,3419,WV,-1.767176,0.419941,0.419941,-0.586476,Vienna,-0.451786,-0.451786,-0.580093,...,-0.032383,"Vienna, West Virginia, Wood County","Vienna, West Virginia",Vienna,WV,West Virginia,Wood,39.3240,-81.5383,"Vienna, West Virginia"


In [161]:
meta_columns = ['city_id', 'State Abbreviation', 'City', 'County Name', 'State', 'County', 'city_state', 'lat', 'lng']
city_metadata = city_all[meta_columns]
city_metadata.to_csv('data/city_metadata.csv', index = False)


In [162]:
def id_to_city(city_id):
    return pd.read_csv('data/city_metadata.csv').iloc[[city_id]]
id_to_city(0)

,city_id,State Abbreviation,City,County Name,State,County,city_state,lat,lng
0,0,AL,Arab,Marshall County,Alabama,Marshall County,"Arab, Alabama",34.3309,-86.4991


In [175]:
city_df = city_all.drop(columns = meta_columns)
delete_cols = ['city_state_id_y', 'state_id', 'city_name_id', 'city', 'state_name', 'county_name']
city_df = city_df.drop(columns = delete_cols)
city_df.head()

,Average Household Income 2020/2021,"Percent Change in Household Income, 1984-2019","Average Yearly Percent Change in Household Income, 1984-2019",MinWage,"Percent Change in Housing Prices, 2000-2019","Average Yearly Percent Change in Housing Prices, 2000-2019",Average House Price 2020/2021,Violent Crime Total,Property Crime Total,Property Crime Per Capita,...,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,-1.244475,0.682348,0.682348,-1.13437,-1.424617,-1.424617,-0.530148,-0.106718,-0.127941,1.084368,...,-0.594826,-0.742922,-1.335937,-1.240936,-0.374992,-0.154439,0.004864,0.328125,0.041230,-0.018375
1,-1.244475,0.682348,0.682348,-1.13437,-0.357814,-0.357814,-0.754485,-0.125265,-0.149551,1.143645,...,-0.596985,-1.132559,-0.880312,-0.555329,-0.708592,0.379059,-0.150118,-0.545845,0.117471,0.980909
2,-1.244475,0.682348,0.682348,-1.13437,-0.324323,-0.324323,-0.237935,-0.070466,0.023579,-0.302775,...,-0.458780,0.425989,-0.465186,-0.974311,-0.977773,-0.483073,1.150718,-1.026759,-0.416213,-0.200487
3,-1.244475,0.682348,0.682348,-1.13437,-0.168164,-0.168164,-0.518289,-0.106718,-0.179453,-0.535400,...,-0.640174,-1.049909,-0.384185,0.619997,0.255398,0.152856,-0.256827,0.184776,-0.797416,-0.256522
4,-1.244475,0.682348,0.682348,-1.13437,-1.612573,-1.612573,-0.756796,-0.119364,-0.174930,0.081101,...,-0.411272,-0.630754,-0.617061,-0.582536,-0.448614,0.639406,-0.254287,-0.943524,0.269952,1.209716


In [176]:
city_columns = list(enumerate(city_df.columns))
city_columns

[(0, 'Average Household Income 2020/2021'),
 (1, 'Percent Change in Household Income, 1984-2019'),
 (2, 'Average Yearly Percent Change in Household Income, 1984-2019'),
 (3, 'MinWage'),
 (4, 'Percent Change in Housing Prices, 2000-2019'),
 (5, 'Average Yearly Percent Change in Housing Prices, 2000-2019'),
 (6, 'Average House Price 2020/2021'),
 (7, 'Violent Crime Total'),
 (8, 'Property Crime Total'),
 (9, 'Property Crime Per Capita'),
 (10, 'Violent Crime Per Capita'),
 (11, 'Total Crime Per Capita'),
 (12, 'TotalPop'),
 (13, 'Men'),
 (14, 'Women'),
 (15, 'VotingAgeCitizen'),
 (16, 'Employed'),
 (17, 'Hispanic'),
 (18, 'White'),
 (19, 'Black'),
 (20, 'Native'),
 (21, 'Asian'),
 (22, 'Pacific'),
 (23, 'Income'),
 (24, 'IncomePerCap'),
 (25, 'Poverty'),
 (26, 'ChildPoverty'),
 (27, 'Professional'),
 (28, 'Service'),
 (29, 'Office'),
 (30, 'Construction'),
 (31, 'Production'),
 (32, 'Drive'),
 (33, 'Carpool'),
 (34, 'Transit'),
 (35, 'Walk'),
 (36, 'OtherTransp'),
 (37, 'WorkAtHome'),
 (

In [177]:
city_np = np.array(city_df)
city_np.shape

(3421, 44)

In [178]:
cities_dist = None
cities_nn = None
if True:
    cities_dist = pairwise_dist(city_np, city_np) #.tofile('data/pairwise_city_dist.csv', sep = ',')
    #np.savetxt("data/city_pairwise_dist.csv", cities_dist, delimiter=",")
    temp = pd.DataFrame(cities_dist)
    temp.to_csv("data/city_distances.csv", index = False)
    cities_nn = pd.DataFrame(np.argsort(cities_dist))
    #np.savetxt("data/city_nn_ranked.csv", cities_nn, delimiter=",")
    cities_nn.to_csv("data/city_nn.csv", index = False)
    cities_dist = temp
    del temp
else:
    cities_dist = np.array(pd.read_csv("data/city_distances.csv"))
    cities_nn = np.array(pd.read_csv("data/city_nn.csv"))

In [180]:
np.sum(cities_nn, axis = 1)

0       5849910
1       5849910
2       5849910
3       5849910
4       5849910
         ...   
3416    5849910
3417    5849910
3418    5849910
3419    5849910
3420    5849910
Length: 3421, dtype: int64

In [182]:
def get_nn(city_id, n = 5):
    """
        Args:
            city_id: int
            n: int
        Return:
            nearest_neighbors: (n,) ndarray, where nearest_neighbors[i] is the city_id of the i'th most similar city
    """

    #Don't include the first column since it will always be itself (distances between a city and itself is 0 miles)
    dta = cities_nn.iloc[city_id, 1:n+1]

    return dta
    
get_nn(0, 5)

1    1276
2    1777
3    1698
4      15
5      25
Name: 0, dtype: int64

In [183]:
cities_nn.head()

,0,1,2,3,4,5,6,7,8,9,...,3411,3412,3413,3414,3415,3416,3417,3418,3419,3420
0,0,1276,1777,1698,15,25,1760,2858,20,1252,...,230,386,2229,214,485,493,1103,3031,382,2121
1,1,1755,1798,2686,2697,1762,2716,2680,1787,2735,...,182,386,2229,485,214,493,1103,3031,382,2121
2,2,18,17,2771,1210,2915,2913,1302,2852,2898,...,386,348,485,214,2229,493,1103,3031,382,2121
3,3,12,11,8,19,2870,2871,2895,2863,2808,...,2860,348,214,485,493,2229,1103,3031,382,2121
4,4,23,6,5,24,9,22,2744,2811,2772,...,348,386,2229,485,214,493,1103,3031,382,2121


In [184]:
city_metadata.columns

Index(['city_id', 'State Abbreviation', 'City', 'County Name', 'State',
       'County', 'city_state', 'lat', 'lng'],
      dtype='object')

In [189]:
nearest_neighbors = pd.concat([city_metadata, cities_nn.iloc[:, 1:11]], axis = 1)

nearest_neighbors['city_county'] = nearest_neighbors.apply(lambda x: x['City'] + ', ' + x['County'], axis = 1)
nearest_neighbors['city_st_abbr'] = nearest_neighbors.apply(lambda x: x['City'] + ', ' + x['State Abbreviation'], axis = 1)

nearest_neighbors.to_csv('data/nearest_neighbors.csv', index = False)

In [192]:
pd.read_csv('data/nearest_neighbors.csv')

,city_id,State Abbreviation,City,County Name,State,County,city_state,lat,lng,1,...,3,4,5,6,7,8,9,10,city_county,city_st_abbr
0,0,AL,Arab,Marshall County,Alabama,Marshall County,"Arab, Alabama",34.3309,-86.4991,1276,...,1698,15,25,1760,2858,20,1252,1803,"Arab, Marshall County","Arab, AL"
1,1,AL,Attalla,Etowah County,Alabama,Etowah County,"Attalla, Alabama",34.0050,-86.1039,1755,...,2686,2697,1762,2716,2680,1787,2735,1252,"Attalla, Etowah County","Attalla, AL"
2,2,AL,Auburn,Lee County,Alabama,Lee County,"Auburn, Alabama",32.6087,-85.4903,18,...,2771,1210,2915,2913,1302,2852,2898,2693,"Auburn, Lee County","Auburn, AL"
3,3,AL,Bay Minette,Baldwin County,Alabama,Baldwin County,"Bay Minette, Alabama",30.8930,-87.7912,12,...,8,19,2870,2871,2895,2863,2808,2923,"Bay Minette, Baldwin County","Bay Minette, AL"
4,4,AL,Chickasaw,Mobile County,Alabama,Mobile County,"Chickasaw, Alabama",30.7715,-88.0793,23,...,5,24,9,22,2744,2811,2772,21,"Chickasaw, Mobile County","Chickasaw, AL"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3416,3416,WV,Princeton,Mercer County,West Virginia,Mercer County,"Princeton, West Virginia",37.3689,-81.0961,3403,...,3414,3411,3419,1288,1309,2881,2882,1310,"Princeton, Mercer County","Princeton, WV"
3417,3417,WV,Shepherdstown,Jefferson County,West Virginia,Jefferson County,"Shepherdstown, West Virginia",39.4320,-77.8046,3406,...,3203,803,847,2709,781,2707,2107,1386,"Shepherdstown, Jefferson County","Shepherdstown, WV"
3418,3418,WV,South Charleston,Kanawha County,West Virginia,Kanawha County,"South Charleston, West Virginia",38.3426,-81.7155,3407,...,3414,3415,3419,3402,18,1212,2837,2704,"South Charleston, Kanawha County","South Charleston, WV"
3419,3419,WV,Vienna,Wood County,West Virginia,Wood County,"Vienna, West Virginia",39.3240,-81.5383,3415,...,1285,1212,1308,1322,3418,2749,1281,3409,"Vienna, Wood County","Vienna, WV"
